In [4]:
import numpy as np
import pandas as pd
import sqlite3
import sys
import os

def clean_host(host):
    return host.split('.')[-2]

def get_site(site_url):
    return site_url.split(".")[1]

def is_third_party(origin, cookie_host):
    if origin in cookie_host:
        return False
    return True

In [7]:
sql_file = 'exp-data/nyt-c1.sqlite' 
    
###### javascript cookies  ######
query = """SELECT sv.site_url, sv.visit_id,
        jsc.host, jsc.name, jsc.value
        FROM javascript_cookies as jsc LEFT JOIN site_visits as sv
        ON sv.visit_id = jsc.visit_id
        """

conn = sqlite3.connect(sql_file)

cookies = pd.read_sql_query(query, conn)

cookies['origin_site'] = list(map(lambda x: get_site(x), cookies.site_url))
cookies["host"] = list(map(lambda x: clean_host(x), cookies.host.tolist()))
cookies["is_third_party"] = list(map(lambda a, b: is_third_party(a, b), cookies.origin_site, cookies.host))

cookies = cookies[["visit_id", "origin_site", "host", "name", "value", "is_third_party"]]
cookies = cookies.drop_duplicates().reset_index(drop = True)

cookies = cookies[cookies.value.str.len() > 5]

In [8]:
# disregard cookie value length < 5 : 539 -> 476
# disregard cookie value in the list  : 476 -> 476

cookies

visit_id origin_site            host       name  \
0           1       webmd           webmd   __cfduid   
1           1       webmd           webmd    lrt_wrk   
2           1       webmd           webmd     gtinfo   
3           1       webmd           webmd  VisitorId   
4           1       webmd           webmd   __cfduid   
..        ...         ...             ...        ...   
532         3     nytimes  rubiconproject      audit   
533         3     nytimes          adsrvr      TDCPM   
534         3     nytimes           yahoo         A3   
536         3     nytimes           yahoo     IDSYNC   
538         3     nytimes        pubmatic         pi   

                                                 value  is_third_party  
0          dd673b2700baa134208e3794c02eec1f41606506583           False  
1    lrt1_cached_k8_worker_1_37S_21O_2020-11-18_17:...           False  
2    {"ct":"Columbus","c":"Franklin","cc":"39049","...           False  
3                 9831e05f-086b-4123-5987-f9b5df875967           False  
4          dc560e081e9789ea111079dc63ad819771606506583           False  
..                                                 ...             ...  
532  1|BZZ8oNzhrJA1Lr8Sne9FWAY6LvqmbcLhYnMP8hKyPQ9I...            True  
533  CAESFgoHcnViaWNvbhILCKz66a-Kxok5EAUSFAoFb3Blbn...            True  
534  d=AQABBIdZwV8CEAePsjTU5cRNE7tmBgFkr9IFEgEBAQGr...            True  
536                                          18y3~1v0j            True  
538                                           156011:2            True  

[476 rows x 6 columns]